In [1]:
#Import the required libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, scale, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Ridge, Lasso
from sklearn import preprocessing


In [2]:
train_data = pd.read_csv('../notebook/cleaned_data_for_regression.csv')
test_data = pd.read_csv('../notebook/cleaned_test_data.csv')

In [3]:
train_data.head()

,Unnamed: 0,amount_donated,control_number,months_since_origin,donor_age,in_house,urbanicity,ses,cluster_code,home_owner,...,lifetime_gift_range,lifetime_max_gift_amt,lifetime_min_gift_amt,last_gift_amt,card_prom_12,number_prom_12,months_since_last_gift,months_since_first_gift,file_avg_gift,file_card_gift
0,0,0.0,5,101,87.0,0,X,5,54,H,...,15.0,20.0,5.0,15.0,5,12,26,92,8.49,7
1,1,10.0,12,137,79.0,0,R,2,45,H,...,20.0,25.0,5.0,17.0,7,21,7,122,14.72,12
2,2,0.0,37,113,75.0,0,S,1,11,H,...,23.0,28.0,5.0,19.0,11,32,6,105,16.75,16
3,3,0.0,38,92,60.0,0,U,2,4,H,...,14.0,17.0,3.0,15.0,11,33,6,92,11.76,12
4,4,0.0,41,101,74.0,0,R,2,49,U,...,20.0,25.0,5.0,25.0,6,19,18,92,8.83,3


In [4]:
def drop_column(column_name, data):
    dropped_data = data.drop([column_name], axis = 1) 
    return dropped_data

In [5]:
train_control_no = train_data.control_number
train_data = drop_column('control_number',train_data)
test_control_no = test_data.control_number
test_data = drop_column('control_number',test_data)

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19372 entries, 0 to 19371
Data columns (total 49 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   19372 non-null  int64  
 1   amount_donated               19372 non-null  float64
 2   months_since_origin          19372 non-null  int64  
 3   donor_age                    19372 non-null  float64
 4   in_house                     19372 non-null  int64  
 5   urbanicity                   19372 non-null  object 
 6   ses                          19372 non-null  int64  
 7   cluster_code                 19372 non-null  int64  
 8   home_owner                   19372 non-null  object 
 9   donor_gender                 19372 non-null  object 
 10  income_group                 19372 non-null  float64
 11  published_phone              19372 non-null  int64  
 12  overlay_source               19372 non-null  object 
 13  mor_hit_rate    

In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148 entries, 0 to 2147
Data columns (total 48 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   2148 non-null   int64  
 1   months_since_origin          2148 non-null   int64  
 2   donor_age                    2148 non-null   float64
 3   in_house                     2148 non-null   int64  
 4   urbanicity                   2148 non-null   object 
 5   ses                          2148 non-null   int64  
 6   cluster_code                 2148 non-null   int64  
 7   home_owner                   2148 non-null   object 
 8   donor_gender                 2148 non-null   object 
 9   income_group                 2148 non-null   float64
 10  published_phone              2148 non-null   int64  
 11  overlay_source               2148 non-null   object 
 12  mor_hit_rate                 2148 non-null   int64  
 13  wealth_rating     

In [8]:
train_data.drop("Unnamed: 0", axis=1, inplace=True)
test_data.drop("Unnamed: 0", axis=1, inplace=True)

In [9]:
train_data.head()

,amount_donated,months_since_origin,donor_age,in_house,urbanicity,ses,cluster_code,home_owner,donor_gender,income_group,...,lifetime_gift_range,lifetime_max_gift_amt,lifetime_min_gift_amt,last_gift_amt,card_prom_12,number_prom_12,months_since_last_gift,months_since_first_gift,file_avg_gift,file_card_gift
0,0.0,101,87.0,0,X,5,54,H,M,2.0,...,15.0,20.0,5.0,15.0,5,12,26,92,8.49,7
1,10.0,137,79.0,0,R,2,45,H,M,7.0,...,20.0,25.0,5.0,17.0,7,21,7,122,14.72,12
2,0.0,113,75.0,0,S,1,11,H,F,5.0,...,23.0,28.0,5.0,19.0,11,32,6,105,16.75,16
3,0.0,92,60.0,0,U,2,4,H,F,6.0,...,14.0,17.0,3.0,15.0,11,33,6,92,11.76,12
4,0.0,101,74.0,0,R,2,49,U,F,2.0,...,20.0,25.0,5.0,25.0,6,19,18,92,8.83,3


In [10]:
test_data.head()

,months_since_origin,donor_age,in_house,urbanicity,ses,cluster_code,home_owner,donor_gender,income_group,published_phone,...,lifetime_gift_range,lifetime_max_gift_amt,lifetime_min_gift_amt,last_gift_amt,card_prom_12,number_prom_12,months_since_last_gift,months_since_first_gift,file_avg_gift,file_card_gift
0,101,59.0,0,R,2,46,U,F,5.0,0,...,16.0,21.0,5.0,21.0,4,10,26,90,14.71,2
1,137,59.0,0,R,2,43,U,F,5.0,1,...,9.0,11.0,2.0,10.0,9,22,22,129,6.00,14
2,17,30.0,0,T,1,35,H,M,6.0,1,...,15.0,20.0,5.0,20.0,5,14,20,87,8.44,1
3,137,75.0,0,U,1,2,H,M,6.0,1,...,5.0,10.0,5.0,10.0,10,24,19,129,6.50,22
4,5,59.0,0,T,2,40,U,F,2.0,0,...,7.0,10.0,3.0,5.0,5,12,18,85,5.83,2


In [11]:
train_data.shape, test_data.shape

((19372, 48), (2148, 47))

In [12]:
train_data = train_data[train_data.amount_donated != 0.00]

In [13]:
train_data.shape

(4843, 48)

In [14]:
# Selecting the independent variables and the target variable
#
y = train_data['amount_donated']
X = train_data.drop('amount_donated', axis = 1)


* **Joining the Train and Test data to encode the categorical columns together**

In [15]:
total_data = pd.concat([X, test_data])

In [16]:
train_data.shape, test_data.shape

((4843, 48), (2148, 47))

In [17]:
categorical_columns = []
for col in total_data.columns:
    unique_column =total_data[col].nunique()
    if unique_column <=60:
        categorical_columns.append(col)

In [18]:
print(categorical_columns)

['months_since_origin', 'in_house', 'urbanicity', 'ses', 'cluster_code', 'home_owner', 'donor_gender', 'income_group', 'published_phone', 'overlay_source', 'mor_hit_rate', 'wealth_rating', 'pct_attribute1', 'pep_star', 'recent_star_status', 'recency_status_96nk', 'frequency_status_97nk', 'recent_card_response_prop', 'recent_response_count', 'recent_card_response_count', 'months_since_last_prom_resp', 'lifetime_card_prom', 'lifetime_min_gift_amt', 'card_prom_12', 'number_prom_12', 'months_since_last_gift', 'file_card_gift']


In [19]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in list(categorical_columns):
        total_data[i]=le.fit_transform(total_data[i])

* **Splitting the Train and Test data to after encoding the categorical columns**

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_d1 = scaler.fit_transform(total_data)
train_d2 = pd.DataFrame(train_d1,columns=total_data.columns)
total_data = train_d2

In [21]:
X_train_cat_encoded =  total_data.iloc[:4843]
X_test_cat_encoded =  total_data.iloc[4843:]
X_train_cat_encoded.shape, X_test_cat_encoded.shape

((4843, 47), (2148, 47))

In [22]:
total_data.head()

,months_since_origin,donor_age,in_house,urbanicity,ses,cluster_code,home_owner,donor_gender,income_group,published_phone,...,lifetime_gift_range,lifetime_max_gift_amt,lifetime_min_gift_amt,last_gift_amt,card_prom_12,number_prom_12,months_since_last_gift,months_since_first_gift,file_avg_gift,file_card_gift
0,1.00,0.903614,0.0,0.2,0.25,0.830189,0.0,0.5,1.000000,0.0,...,0.020060,0.020101,0.357143,0.037778,0.4375,0.395833,0.130435,0.436735,0.029779,0.387097
1,0.55,0.903614,0.0,0.0,0.25,0.509434,0.0,0.0,0.000000,0.0,...,0.004012,0.002010,0.285714,0.011111,0.5625,0.458333,0.217391,0.310204,0.008225,0.258065
2,0.60,0.710843,0.0,0.2,0.25,0.792453,0.0,0.0,0.500000,1.0,...,0.008024,0.006030,0.285714,0.017778,0.6875,0.520833,0.217391,0.310204,0.012170,0.419355
3,1.00,0.674699,0.0,0.2,0.25,0.792453,1.0,0.0,0.666667,1.0,...,0.009027,0.006030,0.232143,0.022222,0.5625,0.416667,0.782609,0.465306,0.010342,0.451613
4,0.50,0.927711,0.0,0.2,0.25,0.830189,1.0,0.5,0.500000,1.0,...,0.010030,0.010050,0.357143,0.033333,0.6875,0.562500,0.086957,0.302041,0.017720,0.225806


In [ ]:
import warnings
warnings.filterwarnings("ignore")



import random
random.seed(1234)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_cat_encoded, y, test_size = 0.30, random_state=0)

import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# # Create linear regression object
clf = linear_model.LinearRegression() 
reg = clf.fit(X_train, y_train)

print('linear model intercept: {}'
     .format(reg.intercept_))

print('linear model coeff:\n {}'
      .format(reg.coef_))

# #train score
scores =  cross_val_score(clf, X_train, y_train,cv=5)
print("R-squared score for training",scores.mean())

#test score

pred_reg = reg.predict(X_test)
scores = cross_val_score(reg, y_test,pred_reg,cv=5)
print("R-squared score for testing", scores.mean())

#MSE
print ("Mean Square error", mean_squared_error(y_test, reg.predict(X_test)))

In [ ]:
random.seed(1234)

# Grid Search for Parameter Tuning 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.model_selection import GridSearchCV

# prepare a range of alpha values to test
k_range = np.array([1,2,3,4,5,6,7,8,9,10])

knn = KNeighborsRegressor(n_neighbors=k_range)
grid = GridSearchCV(knn, param_grid=dict(n_neighbors=k_range), cv= 5)
grid.fit(X_train_cat_encoded, y)
print("grid:",grid)

# summarize the results of the grid search
print("grid.best_score_:",grid.best_score_)
print("grid.best_estimator_:",grid.best_estimator_.n_neighbors)

# Model for best neighbour
model = KNeighborsRegressor(n_neighbors=10)
knn = model.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

# train score
scores =  cross_val_score(model, X_train, y_train,cv=5)
print("R-squared score for training",scores.mean())

#test score
scores = cross_val_score(knn,y_test,knn_pred,cv=5)
print("R-squared score for testing", scores.mean())

#MSE
print ("Mean Square error", mean_squared_error(y_test, knn.predict(X_test)))

In [ ]:
random.seed(1234)

# Grid Search for Algorithm Tuning
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# prepare a range of alpha values to test
alphas = np.array([100,10,1,0.1,0.01,0.001,0.0001,0])

# create and fit a ridge regression model, testing each alpha
ridge_model = Ridge()
grid = GridSearchCV(estimator=ridge_model, param_grid=dict(alpha=alphas), cv=5)

grid.fit(X_train_cat_encoded, y)
print("grid:",grid)

# summarize the results of the grid search
print("grid.best_score_:",grid.best_score_)
print("grid.best_estimator_.alpha:",grid.best_estimator_.alpha)

print('**********************------------------------------****************************')

#  Model for best alpha

model = Ridge(alpha=100)
ridge = model.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)


print('ridge regression linear model intercept: {}'
     .format(ridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(ridge.coef_))

# #train score
scores =  cross_val_score(model, X_train, y_train,cv=5)
print("R-squared score for training",scores.mean())

#test score
scores = cross_val_score(ridge,y_test,ridge_pred,cv=5)
print("R-squared score for testing", scores.mean())

print('Number of non-zero features: {}'
     .format(np.sum(ridge.coef_ != 0)))
#MSE
print ("Mean Square error", mean_squared_error(y_test, ridge.predict(X_test)))

In [ ]:
random.seed(1234)

# Grid Search for parameter Tuning
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# prepare a range of alpha values to test
alphas = np.array([1000,100,10,1,0.1,0.01,0.001,0.0001,0])

# create and fit a ridge regression model, testing each alpha
Lasso_model = Lasso()
grid = GridSearchCV(estimator=Lasso_model, param_grid=dict(alpha=alphas), cv=5)

grid.fit(X_train_cat_encoded, y)
print("grid:",grid)

# summarize the results of the grid search
print("grid.best_score_:",grid.best_score_)
print("grid.best_estimator_.alpha:",grid.best_estimator_.alpha)

print('***********************************************************************')

# Model
model = Lasso(alpha=0.0001,max_iter = 100000)
lasso = model.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)

print('lasso regression linear model intercept: {}'
     .format(lasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(lasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(lasso.coef_ != 0)))

# #train score
scores =  cross_val_score(model, X_train, y_train,cv=5)
print("R-squared score for training",scores.mean())

#test score
scores = cross_val_score(lasso,y_test,lasso_pred,cv=5)
print("R-squared score for testing", scores.mean())

print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X_train), lasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))
        
#MSE
print ("Mean Square error", mean_squared_error(y_test, lasso.predict(X_test)))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline 

df_X = X_train_cat_encoded
df_y = y

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(df_X)

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_poly, df_y, random_state=0)

linreg = LinearRegression().fit(X_train_p, y_train_p)

print('(poly deg 2) linear model coeff (w):\n{}'
     .format(linreg.coef_))
print('(poly deg 2) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train_p, y_train_p)))
print('(poly deg 2) R-squared score (test): {:.3f}\n'
     .format(linreg.score(X_test_p, y_test_p)))

In [ ]:
linreg = Ridge().fit(X_train_p, y_train_p)

print('(poly deg 2 + ridge) linear model coeff (w):\n{}'
     .format(linreg.coef_))
print('(poly deg 2 + ridge) R-squared score (training): {:.3f}'
     .format(linreg.score(X_train_p, y_train_p)))
print('(poly deg 2 + ridge) R-squared score (test): {:.3f}'
     .format(linreg.score(X_test_p, y_test_p)))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn import svm

import warnings
warnings.filterwarnings("ignore")

Cs = [0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
epsilons = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas, 'epsilon' : epsilons}
grid_search = GridSearchCV(svm.SVR(kernel='linear'), param_grid, cv=5)
grid_search.fit(X_train_cat_encoded, y)
grid_search.best_params_
print(grid_search.best_params_)

In [ ]:
#linear SVM model
clf_lin = SVR(kernel='linear',C = 0.01, epsilon = 0.001, gamma = 0.001)
svm_lin = clf_lin.fit(X_train, y_train)
svm_pred = svm_lin.predict(X_test)

# train score
scores =  cross_val_score(clf_lin, X_train, y_train,cv=5)
print("R-squared score for training",scores.mean())
#test score
scores = cross_val_score(svm_lin,y_test,svm_pred,cv=5)
print("R-squared score for testing", scores.mean())
#MSE
print ("Mean Square error", mean_squared_error(y_test, svm_lin.predict(X_test)))

In [ ]:
#Hyper parameter tuning for kernelized SVM
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVR
from sklearn import svm

Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1, 10]
epsilons = [0.01, 0.1, 1, 10]
param_grid = {'C': Cs, 'gamma' : gammas, 'epsilon' : epsilons}
grid_search = GridSearchCV(svm.SVR(kernel='rbf'), param_grid, cv=5)
grid_search.fit(X_train_cat_encoded, y)
grid_search.best_params_
print(grid_search.best_params_)

In [ ]:
#kernelized SVM model
clf_rbf = SVR(kernel='rbf',C = 1, epsilon = 0.01, gamma = 0.01 )

svm_rbf = clf_rbf.fit(X_train, y_train)
svm_pred = svm_rbf.predict(X_test)

# train score
scores =  cross_val_score(clf_rbf, X_train, y_train,cv=5)
print("R-squared score for training",scores.mean())
#test score
scores = cross_val_score(svm_rbf,y_test,svm_pred,cv=5)
print("R-squared score for testing", scores.mean())
#MSE
print ("Mean Square error", mean_squared_error(y_test, svm_rbf.predict(X_test)))